In [22]:
import pymysql
import json
from functools import partial
import csv

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro

def multiple_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        res = cursor.fetchall()
        return res

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)

In [23]:
def add_node(g, n):
    g.add_node(n)
    g.nodes[n]['pos'] = (random(), random())
    g.nodes[n]['size'] = 4
    g.nodes[n]['color'] = (255, 0, 0)

def add_edge(g, n, m):
    g.nodes[n]['size'] += 2
    g.nodes[m]['size'] += 2
    g.add_edge(n, m)
    g.edges[n, m]['color'] = (0, 0, 0, 0.5)

In [34]:
query = 'SELECT * FROM transferencia'
res = multiple_db_results(connection, query)

for r in res:
    if(r[3] = 2019):
        g.add_node()
    print(r)

(1, 2613, 17, 2019)
(2, 2524, 17, 2019)
(3, 3950, 22, 2019)
(4, 3843, 3, 2019)
(5, 97, 5, 2019)
(6, 4049, 28, 2019)
(7, 3575, 27, 2019)
(8, 651, 28, 2019)
(9, 2018, 27, 2019)
(10, 3886, 5, 2019)
(11, 3096, 12, 2019)
(12, 3101, 12, 2019)
(13, 3416, 7, 2019)
(14, 2960, 14, 2019)
(15, 3501, 4, 2019)
(16, 616, 16, 2019)
(17, 2526, 17, 2019)
(18, 1791, 4, 2019)
(19, 1545, 10, 2019)
(20, 556, 25, 2019)
(21, 2600, 15, 2019)
(22, 1372, 31, 2019)
(23, 1499, 3, 2019)
(24, 2244, 21, 2019)
(25, 2240, 18, 2019)
(26, 280, 21, 2019)
(27, 2691, 22, 2019)
(28, 3113, 6, 2019)
(29, 2653, 10, 2019)
(30, 2508, 7, 2019)
(31, 984, 24, 2019)
(32, 4369, 27, 2019)
(33, 913, 9, 2019)
(34, 1500, 30, 2019)
(35, 2832, 27, 2019)
(36, 3321, 14, 2019)
(37, 2242, 4, 2019)
(38, 1658, 24, 2019)
(39, 3938, 23, 2019)
(40, 1498, 2, 2019)
(41, 680, 8, 2019)
(42, 2130, 19, 2019)
(43, 2968, 8, 2019)
(44, 4155, 3, 2019)
(45, 2443, 28, 2019)
(46, 2176, 27, 2019)
(47, 2129, 24, 2019)
(48, 1799, 23, 2019)
(49, 550, 25, 2019)
(50, 

In [13]:
connection.close()

In [ ]:
a = fm.Animation()

g = fm.Graph(nx.Graph())

g.add_node()
a.rec(g)

add_node(g, 1)
add_edge(g, 0, 1)
g.move('step')
a.rec(g)

for n in range(2, NUM_NODES):
    nodes = list(g.nodes)
    weights = [g.degree(m) for m in g.nodes]
    targets = choices(nodes, weights)
    m = targets[0]

    add_node(g, n)
    add_edge(g, n, m)
    g.move('step')
    a.rec(g)

a.play()